In [130]:
import pandas as pd
import zipcodes
import requests
import datetime as dt

In [5]:
df_bluebikes = pd.read_csv('202302-bluebikes-tripdata.csv')
df_bluebikes

# filter out stations by postal code
# 30 minute intervals
df_bluebikes

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,postal code
0,263,2023-02-01 00:00:50.0410,2023-02-01 00:05:13.7090,330,30 Dane St,42.381001,-71.104025,110,Harvard University Gund Hall at Quincy St / Ki...,42.376369,-71.114025,7334,Subscriber,02138
1,447,2023-02-01 00:02:50.1390,2023-02-01 00:10:17.4090,413,Kennedy-Longfellow School 158 Spring St,42.369553,-71.085790,386,Sennott Park Broadway at Norfolk Street,42.368605,-71.099302,3257,Subscriber,02139
2,302,2023-02-01 00:09:00.7270,2023-02-01 00:14:03.2560,554,Forsyth St at Huntington Ave,42.339202,-71.090511,27,Roxbury Crossing T Stop - Columbus Ave at Trem...,42.331184,-71.095171,7824,Subscriber,02120
3,689,2023-02-01 00:16:00.6020,2023-02-01 00:27:30.4070,87,Harvard University Housing - 115 Putnam Ave at...,42.366621,-71.114214,178,MIT Pacific St at Purrington St,42.359573,-71.101295,8266,Subscriber,02139
4,468,2023-02-01 00:18:10.3820,2023-02-01 00:25:59.3730,55,Boylston St at Massachusetts Ave,42.347406,-71.086784,32,Landmark Center - Brookline Ave at Park Dr,42.343691,-71.102353,7431,Subscriber,02215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152970,417,2023-02-28 23:57:36.9220,2023-03-01 00:04:34.5830,46,Christian Science Plaza - Massachusetts Ave at...,42.343666,-71.085824,331,Huntington Ave at Mass Art,42.336586,-71.098870,3442,Subscriber,02120
152971,872,2023-02-28 23:58:06.8950,2023-03-01 00:12:39.3410,11,Longwood Ave at Binney St,42.338629,-71.106500,491,Harrison Ave at E. Dedham St,42.339194,-71.069750,2876,Subscriber,02446
152972,384,2023-02-28 23:58:20.9490,2023-03-01 00:04:45.2180,553,Cambridge Crossing at North First Street,42.371141,-71.076198,413,Kennedy-Longfellow School 158 Spring St,42.369553,-71.085790,6349,Subscriber,02141
152973,593,2023-02-28 23:58:36.5470,2023-03-01 00:08:29.6560,36,Copley Square - Dartmouth St at Boylston St,42.349928,-71.077392,152,Ink Block - Harrison Ave at Herald St,42.345901,-71.063187,8277,Subscriber,33332


In [30]:
df_cur_stations = pd.read_csv('current_bluebikes_stations.csv', header=1)
df_cur_stations

,Number,Name,Latitude,Longitude,District,Public,Total docks,Deployment Year
0,K32015,1200 Beacon St,42.344149,-71.114674,Brookline,Yes,15,2021.0
1,W32006,160 Arsenal,42.364664,-71.175694,Watertown,Yes,11,2021.0
2,A32019,175 N Harvard St,42.363796,-71.129164,Boston,Yes,17,2014.0
3,S32035,191 Beacon St,42.380323,-71.108786,Somerville,Yes,19,2018.0
4,C32094,2 Hummingbird Lane at Olmsted Green,42.288870,-71.095003,Boston,Yes,17,2020.0
...,...,...,...,...,...,...,...,...
443,N32005,West Newton,42.349601,-71.226275,Newton,Yes,14,2020.0
444,A32043,Western Ave at Richardson St,42.361787,-71.143931,Boston,Yes,0,2019.0
445,B32059,Whittier St Health Center,42.332863,-71.092189,Boston,Yes,19,2019.0
446,D32040,Williams St at Washington St,42.306539,-71.107669,Boston,Yes,23,2018.0


In [17]:
list(df_bluebikes['end station name'].values)

set_names = set()

for name in df_bluebikes['end station name'].values:
    if name in df_cur_stations['Name'].values:
        pass
    else:
        set_names.add(name)

In [18]:
set_names

{'18 Dorrance Warehouse',
 '87-101 Cambridgepark Drive',
 'Boylston St at Charles St (Temp Winter Location)',
 'Boynton Yards at 101 South Street',
 'Broadway T Stop',
 'Graham and Parks School – Linnaean St at Walker St',
 'Linear Park - Mass. Ave. at Cameron Ave. '}

In [31]:
#df_bluebikes.loc[df_bluebikes['start station name'] == 'Bolyston St at Charles St']
df_cur_stations['Name'] = df_cur_stations['Name'].replace('Graham and Parks School ? Linnaean St at Walker St',
                                          'Graham and Parks School - Linnaean St at Walker St')
df_cur_stations['Name'] = df_cur_stations['Name'].replace('Linear Park - Mass. Ave. at Cameron Ave.',
                                          'Linear Park - Mass. Ave. at Cameron Ave. ')
df_cur_stations['Name'] = df_cur_stations['Name'].replace('Boylston St at Charles St (Temp Winter Location)',
                                          'Boylston St at Charles St')
df_cur_stations

In [32]:
df_cur_stations

,Number,Name,Latitude,Longitude,District,Public,Total docks,Deployment Year
0,K32015,1200 Beacon St,42.344149,-71.114674,Brookline,Yes,15,2021.0
1,W32006,160 Arsenal,42.364664,-71.175694,Watertown,Yes,11,2021.0
2,A32019,175 N Harvard St,42.363796,-71.129164,Boston,Yes,17,2014.0
3,S32035,191 Beacon St,42.380323,-71.108786,Somerville,Yes,19,2018.0
4,C32094,2 Hummingbird Lane at Olmsted Green,42.288870,-71.095003,Boston,Yes,17,2020.0
...,...,...,...,...,...,...,...,...
443,N32005,West Newton,42.349601,-71.226275,Newton,Yes,14,2020.0
444,A32043,Western Ave at Richardson St,42.361787,-71.143931,Boston,Yes,0,2019.0
445,B32059,Whittier St Health Center,42.332863,-71.092189,Boston,Yes,19,2019.0
446,D32040,Williams St at Washington St,42.306539,-71.107669,Boston,Yes,23,2018.0


In [45]:
filtered_zips = zipcodes.filter_by(city="Boston", state="MA") 
filtered_zips

boston_zips_list = []

for i in filtered_zips:
    boston_zips_list.append(i['zip_code'])
    
boston_zips_list

['02108',
 '02109',
 '02110',
 '02111',
 '02112',
 '02113',
 '02114',
 '02115',
 '02116',
 '02117',
 '02118',
 '02123',
 '02133',
 '02163',
 '02196',
 '02199',
 '02201',
 '02203',
 '02204',
 '02205',
 '02206',
 '02207',
 '02210',
 '02211',
 '02212',
 '02215',
 '02216',
 '02217',
 '02222',
 '02241',
 '02266',
 '02283',
 '02284',
 '02293',
 '02295',
 '02297',
 '02298']

In [47]:
df_bluebikes = df_bluebikes.loc[df_bluebikes['postal code'].isin(boston_zips_list)]
df_bluebikes

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,postal code
4,468,2023-02-01 00:18:10.3820,2023-02-01 00:25:59.3730,55,Boylston St at Massachusetts Ave,42.347406,-71.086784,32,Landmark Center - Brookline Ave at Park Dr,42.343691,-71.102353,7431,Subscriber,02215
8,1062,2023-02-01 00:27:12.8890,2023-02-01 00:44:55.4800,24,Seaport Square - Seaport Blvd at Northern Ave,42.351482,-71.044361,58,Mugar Way at Beacon St,42.355536,-71.072869,3085,Subscriber,02118
14,1070,2023-02-01 00:38:58.6470,2023-02-01 00:56:48.9690,30,Brigham Circle - Francis St at Huntington Ave,42.334629,-71.104079,59,Chinatown Gate Plaza,42.351356,-71.059367,8444,Subscriber,02116
15,686,2023-02-01 00:47:09.2450,2023-02-01 00:58:35.8090,361,Deerfield St at Commonwealth Ave,42.349244,-71.097282,9,Commonwealth Ave at Agganis Way,42.351692,-71.119035,5356,Subscriber,02215
20,321,2023-02-01 00:54:09.3540,2023-02-01 00:59:30.4490,558,St. Alphonsus St at Tremont St,42.333293,-71.101246,3,Colleges of the Fenway - Fenway at Avenue Loui...,42.340115,-71.100619,4879,Subscriber,02215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152939,349,2023-02-28 23:21:39.2690,2023-02-28 23:27:28.9750,554,Forsyth St at Huntington Ave,42.339202,-71.090511,27,Roxbury Crossing T Stop - Columbus Ave at Trem...,42.331184,-71.095171,3743,Subscriber,02118
152946,1086,2023-02-28 23:30:26.3020,2023-02-28 23:48:32.3410,68,Central Square at Mass Ave / Essex St,42.365070,-71.103100,14,HMS/HSPH - Avenue Louis Pasteur at Longwood Ave,42.337417,-71.102861,8473,Subscriber,02115
152954,855,2023-02-28 23:37:46.0160,2023-02-28 23:52:02.0100,9,Commonwealth Ave at Agganis Way,42.351692,-71.119035,9,Commonwealth Ave at Agganis Way,42.351692,-71.119035,6736,Subscriber,02215
152961,477,2023-02-28 23:48:25.7650,2023-02-28 23:56:22.7820,60,Charles Circle - Charles St at Cambridge St,42.360793,-71.071190,59,Chinatown Gate Plaza,42.351356,-71.059367,3820,Subscriber,02116


In [48]:
df_cur_stations = df_cur_stations.loc[df_cur_stations['District'] == 'Boston']
df_cur_stations

,Number,Name,Latitude,Longitude,District,Public,Total docks,Deployment Year
2,A32019,175 N Harvard St,42.363796,-71.129164,Boston,Yes,17,2014.0
4,C32094,2 Hummingbird Lane at Olmsted Green,42.288870,-71.095003,Boston,Yes,17,2020.0
8,C32106,555 Metropolitan Ave,42.268100,-71.119240,Boston,Yes,18,2021.0
9,C32105,606 American Legion Hwy at Canterbury St,42.285780,-71.109725,Boston,Yes,18,2021.0
10,C32091,645 Summer St,42.341781,-71.039870,Boston,Yes,19,2020.0
...,...,...,...,...,...,...,...,...
441,B32021,Wentworth Institute of Technology - Huntington...,42.337586,-71.096271,Boston,Yes,14,2015.0
442,D32022,West End Park,42.365908,-71.064467,Boston,Yes,35,2012.0
444,A32043,Western Ave at Richardson St,42.361787,-71.143931,Boston,Yes,0,2019.0
445,B32059,Whittier St Health Center,42.332863,-71.092189,Boston,Yes,19,2019.0


In [50]:
df_bluebikes.to_csv('boston_202303_bluebikes_tripdata.csv')
df_cur_stations.to_csv('boston_current_bluebike_stations.csv')

In [77]:
request = requests.get("https://gbfs.bluebikes.com/gbfs/en/station_information.json")
request.json()

{'data': {'stations': [{'rental_methods': ['KEY', 'CREDITCARD'],
    'capacity': 15,
    'external_id': 'f83464e4-0de8-11e7-991c-3863bb43a7d0',
    'eightd_has_key_dispenser': True,
    'lat': 42.34011512249236,
    'short_name': 'B32006',
    'has_kiosk': True,
    'station_id': '3',
    'legacy_id': '3',
    'lon': -71.10061883926392,
    'region_id': '10',
    'name': 'Colleges of the Fenway - Fenway at Avenue Louis Pasteur',
    'station_type': 'classic',
    'electric_bike_surcharge_waiver': False,
    'eightd_station_services': []},
   {'rental_methods': ['KEY', 'CREDITCARD'],
    'capacity': 19,
    'external_id': 'f834658f-0de8-11e7-991c-3863bb43a7d0',
    'eightd_has_key_dispenser': False,
    'lat': 42.345392,
    'short_name': 'C32000',
    'has_kiosk': True,
    'station_id': '4',
    'legacy_id': '4',
    'lon': -71.069616,
    'region_id': '10',
    'name': 'Tremont St at E Berkeley St',
    'station_type': 'classic',
    'electric_bike_surcharge_waiver': False,
    'eigh

In [86]:
request.json()['data']['stations'][1]

{'rental_methods': ['KEY', 'CREDITCARD'],
 'capacity': 19,
 'external_id': 'f834658f-0de8-11e7-991c-3863bb43a7d0',
 'eightd_has_key_dispenser': False,
 'lat': 42.345392,
 'short_name': 'C32000',
 'has_kiosk': True,
 'station_id': '4',
 'legacy_id': '4',
 'lon': -71.069616,
 'region_id': '10',
 'name': 'Tremont St at E Berkeley St',
 'station_type': 'classic',
 'electric_bike_surcharge_waiver': False,
 'eightd_station_services': []}

In [88]:
kiosk_list = []
for i in request.json()['data']['stations']:
    kiosk_list.append(i['has_kiosk'])
    
kiosk_list

AttributeError: 'list' object has no attribute 'unique'

In [91]:
request2 = requests.get("https://gbfs.bluebikes.com/gbfs/fr/free_bike_status.json")
request2.json()

{'data': {'bikes': []}, 'last_updated': 1680644220, 'ttl': 5}

In [76]:
request3 = requests.get("https://gbfs.bluebikes.com/gbfs/en/system_calendar.json")
request3.json()

{'data': {'calendars': [{'start_day': 1,
    'end_day': 31,
    'start_month': 1,
    'end_month': 12}]},
 'last_updated': 1680644025,
 'ttl': 5}

In [103]:
datetime.strptime('2023-02-01 00:18:10.3820', '%Y-%m-%d %H:%M:%S.%f')

datetime.datetime(2023, 2, 1, 0, 18, 10, 382000)

In [131]:
df_bluebikes['starttime'] = df_bluebikes['starttime'].apply(lambda x : dt.datetime.strptime(x, 
                                              '%Y-%m-%d %H:%M:%S.%f'))
df_bluebikes['stoptime'] = df_bluebikes['stoptime'].apply(lambda x : dt.datetime.strptime(x, 
                                              '%Y-%m-%d %H:%M:%S.%f'))

TypeError: strptime() argument 1 must be str, not Timestamp

In [134]:
df_bluebikes['day_of_week'] = df_bluebikes['starttime'].dt.dayofweek

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [135]:
df_bluebikes

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,postal code,day_of_week
4,468,2023-02-01 00:18:10.382,2023-02-01 00:25:59.373,55,Boylston St at Massachusetts Ave,42.347406,-71.086784,32,Landmark Center - Brookline Ave at Park Dr,42.343691,-71.102353,7431,Subscriber,02215,2
8,1062,2023-02-01 00:27:12.889,2023-02-01 00:44:55.480,24,Seaport Square - Seaport Blvd at Northern Ave,42.351482,-71.044361,58,Mugar Way at Beacon St,42.355536,-71.072869,3085,Subscriber,02118,2
14,1070,2023-02-01 00:38:58.647,2023-02-01 00:56:48.969,30,Brigham Circle - Francis St at Huntington Ave,42.334629,-71.104079,59,Chinatown Gate Plaza,42.351356,-71.059367,8444,Subscriber,02116,2
15,686,2023-02-01 00:47:09.245,2023-02-01 00:58:35.809,361,Deerfield St at Commonwealth Ave,42.349244,-71.097282,9,Commonwealth Ave at Agganis Way,42.351692,-71.119035,5356,Subscriber,02215,2
20,321,2023-02-01 00:54:09.354,2023-02-01 00:59:30.449,558,St. Alphonsus St at Tremont St,42.333293,-71.101246,3,Colleges of the Fenway - Fenway at Avenue Loui...,42.340115,-71.100619,4879,Subscriber,02215,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152939,349,2023-02-28 23:21:39.269,2023-02-28 23:27:28.975,554,Forsyth St at Huntington Ave,42.339202,-71.090511,27,Roxbury Crossing T Stop - Columbus Ave at Trem...,42.331184,-71.095171,3743,Subscriber,02118,1
152946,1086,2023-02-28 23:30:26.302,2023-02-28 23:48:32.341,68,Central Square at Mass Ave / Essex St,42.365070,-71.103100,14,HMS/HSPH - Avenue Louis Pasteur at Longwood Ave,42.337417,-71.102861,8473,Subscriber,02115,1
152954,855,2023-02-28 23:37:46.016,2023-02-28 23:52:02.010,9,Commonwealth Ave at Agganis Way,42.351692,-71.119035,9,Commonwealth Ave at Agganis Way,42.351692,-71.119035,6736,Subscriber,02215,1
152961,477,2023-02-28 23:48:25.765,2023-02-28 23:56:22.782,60,Charles Circle - Charles St at Cambridge St,42.360793,-71.071190,59,Chinatown Gate Plaza,42.351356,-71.059367,3820,Subscriber,02116,1


In [136]:
df_bluebikes.to_csv('boston_bluebikes_202303_datetimes.csv')